In [1]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("nechbamohammed/research-papers-dataset")

print("Path to dataset files:", path)
import os
csv_file=os.listdir(path)
print(csv_file[0])

csv_file_path = os.path.join(path, csv_file[0])
print(csv_file_path)

/opt/anaconda3/envs/py_proj/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


100%|██████████| 590M/590M [02:22<00:00, 4.34MB/s] 

Extracting files...


Path to dataset files: /Users/anuragprasad/.cache/kagglehub/datasets/nechbamohammed/research-papers-dataset/versions/1
dblp-v10.csv
/Users/anuragprasad/.cache/kagglehub/datasets/nechbamohammed/research-papers-dataset/versions/1/dblp-v10.csv


In [2]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import json

In [3]:
df=pd.read_csv(csv_file_path)

In [4]:
print(df.head(10))

                                            abstract  \
0  In this paper, a robust 3D triangular mesh wat...   
1  We studied an autoassociative neural network w...   
2  It is well-known that Sturmian sequences are t...   
3  One of the fundamental challenges of recognizi...   
4  This paper generalizes previous optimal upper ...   
5  Queueing networks with multiple classes of cus...   
6  A CMOS oscillator employing differential trans...   
7  In statistical setting of the pattern recognit...   
8  We first discuss some general aspects of micro...   
9                                                NaN   

                                             authors  n_citation  \
0             ['S. Ben Jabra', 'Ezzeddine Zagrouba']          50   
1  ['Joaquín J. Torres', 'Jesús M. Cortés', 'Joaq...          50   
2           ['Genevi eve Paquin', 'Laurent Vuillon']          50   
3  ['Yaser Sheikh', 'Mumtaz Sheikh', 'Mubarak Shah']         221   
4  ['Efraim Laksman', 'Håkan Lennerstad', '

In [5]:
df['combined_text']=df['title'].fillna('') +df['abstract'].fillna('')

In [6]:
# df=df.head(10)

In [7]:
tfidf=TfidfVectorizer(stop_words="english")
tfidf_matrix=tfidf.fit_transform(df['combined_text'])

In [8]:
cosineSimilarity=cosine_similarity(tfidf_matrix,tfidf_matrix)

In [16]:
def getPaperIndex(title):
    return df[df['title'].str.strip().str.lower() == title.strip().lower()].index[0]

In [17]:
def get_paper_recommendations(paper_title, cosine_sim=cosineSimilarity, df=df):
    idx=getPaperIndex(paper_title)
    
    #For a given paper calculates its cosine similarity with respect to all the papers
    similar_scores=list(enumerate(cosineSimilarity[idx]))
    
    #Then sorts all the papers in the decreasing(reverse) order of their 
    #Cosine Similarity with the 'idx' index paper
    
    similar_scores=sorted(similar_scores, key=lambda x:x[1], reverse=True)
    
    #Then returns the 5 most similar papers to the user apart from the 
    #paper itself and recommends it to the user
    
    similar_scores=similar_scores[1:6]
    
    
    #To get the indices of the most similar papers:
    paper_indices=[i[0] for i in similar_scores]

    #Then we create a list of dictionaries that contain all the info
    #about the recommended papers from

    #The orient='records' converts each row to a different dictionary
    recommended_papers=df.iloc[paper_indices][['title','authors','year', 'references','n_citation','venue']].to_dict(orient='records')
    
    return json.dumps(recommended_papers, indent=5)

In [22]:
input_paper="A new approach of 3D watermarking based on image segmentation"
    
recommended_papers=get_paper_recommendations(input_paper)
print(recommended_papers)

[
     {
          "title": "Exploring the space of a human action",
          "authors": "['Yaser Sheikh', 'Mumtaz Sheikh', 'Mubarak Shah']",
          "year": 2005,
          "references": "['056116c1-9e7a-4f9b-a918-44eb199e67d6', '05ac52a1-dfa2-4c32-9a54-6c1e58cfcca5', '0b2f71cc-6797-445a-82df-5be29945c0a7', '60cf9842-c3aa-463d-a5cf-17fbb8f99ccd', '7a3bb575-8313-4e77-a008-ee5cde444746', '7eda9b91-21f1-490f-8ef2-f94e81c558a4', '905461e4-643b-4da0-a669-f52318b9e126', 'be336575-9974-411d-b637-45440213cbd4', 'be9527aa-97e2-4951-86d1-cc6130cbb777', 'f0d57def-3340-49be-a1c3-cbc996af8dec']",
          "n_citation": 221,
          "venue": "international conference on computer vision"
     },
     {
          "title": "A Push\u2013Pull Class-C CMOS VCO",
          "authors": "['Andrea Mazzanti', 'Pietro Andreani']",
          "year": 2013,
          "references": "['0a09db01-264a-4bdf-942c-d33cceb35d3c', '36c942df-7fcb-40b8-b023-a3a4001f632e', '84c00b06-a611-4082-89f7-df2d5540d210', 'a54965